In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Local modules
import data_organizer as do
from calculator import Calculator, cornell_potential_ansatz

# Setup plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# --- CONFIGURATION ---

# 1. Data Path 
FILE_PATH = "../data/20260205/36"

# 2. Analysis Parameters
THERMALIZATION_STEPS = 500
SOMMER_TARGET_FORCE = 1.65

print(f"Analyzing Directory: {FILE_PATH}")
print(f"Parameters: Therm={THERMALIZATION_STEPS}, Target={SOMMER_TARGET_FORCE}")

# Potential V(R)

In [ ]:
# --- 1. Load Data and Initialize Calculator ---

try:
    experiment = do.ExperimentData(FILE_PATH)
    
    if "W_temp" not in experiment.data or not experiment.data["W_temp"]:
        print("Error: No 'W_temp.out' file found in the directory.")
    else:
        # Get the first W_temp file (assuming only one usually or we take the first)
        w_temp_data = experiment.data["W_temp"][0]
        print(f"Loaded W_temp data with {len(w_temp_data.observables[0].values)} entries (after thermalization cut if applied).")
        calc = Calculator(w_temp_data, n_bootstrap=50, step_size=1)
        print("Calculator initialized.")

except Exception as e:
    print(f"An error occurred loading data: {e}")


In [ ]:
# --- 2. Plot W(R, T) vs T (Step 1: Extracting V(R)) ---

# Identify available R values (L column in W_temp)
try:
    all_L = np.array(w_temp_data.get("L").values)
    unique_Rs = sorted(np.unique(all_L))
    
    # Pick a few representative R values to plot
    # e.g., smallest, middle, largest
    r_to_plot = [unique_Rs[0], unique_Rs[len(unique_Rs)//2], unique_Rs[-1]]
    if len(unique_Rs) > 3:
        # Maybe pick explicitly R=2, 4, 6 if available
        candidates = [2, 4, 6, 8]
        r_to_plot = [r for r in candidates if r in unique_Rs]
        if not r_to_plot: r_to_plot = unique_Rs[:3]

    # Define a set of t_min values to compare the fit stability
    t_min_values = np.arange(1, 10)
    
    print(f"Plotting W(R,T) fits for R = {r_to_plot}")
    print(f"Comparing fits for t_min = {t_min_values}")

    fig, axes = plt.subplots(1, len(r_to_plot), figsize=(5*len(r_to_plot), 5), constrained_layout=True)
    if len(r_to_plot) == 1: axes = [axes]

    for ax, r_val in zip(axes, r_to_plot):
        # 1. Get W(R, T) data points
        # We need to query T values for this R
        all_T = np.array(w_temp_data.get("T").values)
        unique_Ts = sorted(np.unique(all_T[all_L == r_val]))
        
        ws, ws_err, ts = [], [], []
        
        for t_val in unique_Ts:
            try:
                # W_R_T calculator retrieves the mean and error for W(R,T)
                w_var = calc.get_variable("W_R_T", R=r_val, T=t_val)
                ws.append(w_var.get())
                ws_err.append(w_var.err() if w_var.err() else 0)
                ts.append(t_val)
            except Exception:
                continue
                
        # Filter negative or zero values for log plot
        ts = np.array(ts)
        ws = np.array(ws)
        ws_err = np.array(ws_err)
        
        mask = ws > 0
        ts_fit = ts[mask]
        ws_fit = ws[mask]
        ws_err_fit = ws_err[mask]
        
        # Plot Data Points
        ax.errorbar(ts_fit, ws_fit, yerr=ws_err_fit, fmt='o', color='black', alpha=0.3, label=f'Data R={r_val}')
        
        # 2. Get V(R) results from Calculator for multiple t_min values
        colors = plt.cm.viridis(np.linspace(0, 0.9, len(t_min_values)))
        
        for idx, t_start in enumerate(t_min_values):
            try:
                # We reuse the same calculator instance but request different t_min
                v_var = calc.get_variable("V_R", R=r_val, t_min=t_start)
                v_val = v_var.get()
                
                # Retrieve C directly from calculator
                C_est = v_var.parameters.get("fit_C", None)
                
                if not np.isnan(v_val):
                    if C_est is None:
                        # Fallback if fit_C is missing
                        mask_t = ts_fit >= t_start
                        if np.any(mask_t):
                            ln_ws = np.log(ws_fit[mask_t])
                            ts_subset = ts_fit[mask_t]
                            ln_C_vals = ln_ws + v_val * ts_subset
                            ln_C = np.mean(ln_C_vals)
                            C_est = np.exp(ln_C)
                        else:
                            C_est = 1.0

                    # Generate line points
                    t_line = np.linspace(min(ts_fit), max(ts_fit), 100)
                    w_line = C_est * np.exp(-v_val * t_line)
                    
                    ax.plot(t_line, w_line, '-', color=colors[idx], label=f't_min={t_start}: V={v_val:.4f}')
                    
                    # Highlight fit region start
                    # ax.axvline(t_start, color=colors[idx], linestyle=':', alpha=0.5)

            except Exception as e:
                print(f"Could not get V_R for R={r_val}, t_min={t_start}: {e}")

        ax.set_yscale('log')
        ax.set_title(f"Wilson Loop Decay (R={r_val})")
        ax.set_xlabel("T")
        ax.set_ylabel("W(R,T)")
        ax.legend()

    plt.show()

except Exception as e:
    print(f"Error in plotting W(R,T): {e}")

In [ ]:
try:
    fig, ax = plt.subplots(figsize=(10, 6))

    # Compare a few t_min values to check for plateau stability
    comparison_t_mins = t_min_values

    for t_start in comparison_t_mins:
        rs_plot, vs_plot, err_plot = [], [], []
        
        for r_val in unique_Rs:
            try:
                # Extract V(R) for this specific t_min
                v_var = calc.get_variable("V_R", R=r_val, t_min=t_start)
                val = v_var.get()
                
                if not np.isnan(val):
                    rs_plot.append(r_val)
                    vs_plot.append(val)
                    err_plot.append(v_var.err() if v_var.err() is not None else 0)
            except Exception:
                continue
        
        ax.errorbar(rs_plot, vs_plot, yerr=err_plot, fmt='o-', capsize=3, label=f'$t_{{min}}={t_start}$')

    ax.set_title("Static Potential $V(R)$ Stability (Comparison of $t_{min}$)")
    ax.set_xlabel("$R/a$")
    ax.set_ylabel("$aV(R)$")
    ax.legend()
    plt.show()

except Exception as e:
    print(f"Error comparing V(R) fits: {e}")

    
try:
    target_R = 6
    if target_R in unique_Rs:
        t_mins_r6, vs_r6, errs_r6 = [], [], []
        
        for t_start in t_min_values:
            try:
                # Extract V(R) for the target R and current t_min
                v_var = calc.get_variable("V_R", R=target_R, t_min=t_start)
                val = v_var.get()
                    
                if not np.isnan(val):
                    t_mins_r6.append(t_start)
                    vs_r6.append(val)
                    errs_r6.append(v_var.err() if v_var.err() is not None else 0)
            except Exception:
                continue
            
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.errorbar(t_mins_r6, vs_r6, yerr=errs_r6, fmt='s-', capsize=4, color='firebrick', label=f'$R={target_R}$')
           
        ax.set_title(f"Plateau Stability: $V(R={target_R})$ vs. $t_{{min}}$")
        ax.set_xlabel("$t_{min}$")
        ax.set_ylabel(f"$aV(R={target_R})$")
        ax.legend()
        plt.show()
    else:
        print(f"R={target_R} not found in unique_Rs.")

except Exception as e:
    print(f"Error plotting V(R=6) stability: {e}")



In [ ]:
# --- 4. Plot V(R) vs R and Cornell Fit (Step 2: Calculating r0) ---
R0_T_MIN = 5
try:
    # 1. Calculate r0 and get fit parameters
    # The default t_min in calculator is 2, but we use R0_T_MIN from config
    r0_var = calc.get_variable("r0", t_min=R0_T_MIN, target_force=SOMMER_TARGET_FORCE)
    r0_val = r0_var.get()
    
    # Extract Cornell Parameters from the result
    # The variable stores these in its parameters dict
    cornell_params = r0_var.parameters.get("cornell_params", None)
    
    print(f"Calculated r0: {r0_val}")
    if cornell_params:
        print(f"Cornell Parameters: {cornell_params}")
    else:
        print("Warning: Cornell parameters not found in r0 variable.")

    # 2. Collect V(R) points for plotting
    # We must use the same t_min as used in r0 calculation
    rs = []
    vs = []
    vs_err = []
    
    for r_val in unique_Rs:
        try:
            v_var = calc.get_variable("V_R", R=r_val, t_min=R0_T_MIN)
            val = v_var.get()
            err = v_var.err()
            
            if not np.isnan(val):
                rs.append(r_val)
                vs.append(val)
                vs_err.append(err if err is not None else 0)
        except Exception:
            continue
            
    # 3. Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot V(R) Points
    ax.errorbar(rs, vs, yerr=vs_err, fmt='o', label='V(R) Data', zorder=5)
    # out raw data
    print(f"V(R) data points: {list(zip(rs, vs))}")
    # Plot Cornell Fit Curve
    if cornell_params:
        A = cornell_params['A']
        sigma = cornell_params['sigma']
        B = cornell_params['B']
        
        # Smooth line for plotting
        r_line = np.linspace(min(rs), max(rs), 100)
        v_line = cornell_potential_ansatz(r_line, A, sigma, B)
        
        ax.plot(r_line, v_line, 'r-', label=f'Cornell Fit\nA={A:.2f}, sig={sigma:.2f}, B={B:.2f}')
        
        # Mark r0
        if not np.isnan(r0_val):
            # Calculate V(r0)
            v_r0 = cornell_potential_ansatz(r0_val, A, sigma, B)
            ax.plot(r0_val, v_r0, 'ks', markersize=8, label=f'$r_0/a$ = {r0_val:.3f}')
            # Draw lines to axes
            ax.axvline(r0_val, color='k', linestyle=':', alpha=0.5)

    ax.set_title(f"Static Potential V(R) and Cornell Fit (t_min={R0_T_MIN})")
    ax.set_xlabel("R/a")
    ax.set_ylabel("aV(R)")
    ax.legend()
    plt.show()

except Exception as e:
    print(f"Error in plotting V(R): {e}")
